In [12]:
!pip install transformers

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import AdamW 
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np

In [2]:
# Load the dataset
data = pd.read_excel('/kaggle/input/data-set/5247-rows_3-Emotions_No-Type.xlsx')

In [3]:
# Extract input (Utterance) and target (Emotion)
X = data['Utterance'].values
y = data['Emotion'].values

In [4]:
# Label encode 'Emotion' with new values [-1, 0, 1]
label_encoder_emotion = LabelEncoder()
y = label_encoder_emotion.fit_transform(y)

In [5]:
# Label encode 'Dialogue_Act'
label_encoder_dialogue_act = LabelEncoder()
dialogue_act_encoded = label_encoder_dialogue_act.fit_transform(data['Dialogue_Act'])

In [6]:
# Set device to GPU if available, otherwise CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
# Tokenize the data
def tokenize_data(text_list):
    return tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

In [9]:
# PyTorch Dataset Class
class EmotionDataset(Dataset):
    def __init__(self, encodings, dialogue_act, labels):
        self.encodings = encodings
        self.dialogue_act = dialogue_act
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['dialogue_act'] = torch.tensor(self.dialogue_act[idx], dtype=torch.long)
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
# Custom BERT Model with Embeddings for Dialogue Act
class BertWithAdditionalFeatures(nn.Module):
    def __init__(self, bert_model, dialogue_act_vocab_size, embedding_dim, num_labels):
        super(BertWithAdditionalFeatures, self).__init__()
        self.bert = bert_model
        self.dialogue_act_embedding = nn.Embedding(dialogue_act_vocab_size, embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(bert_model.config.hidden_size + embedding_dim, num_labels)

    def forward(self, input_ids, attention_mask, dialogue_act):
        # Get BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]

        # Get embedding for Dialogue Act
        dialogue_act_embedded = self.dialogue_act_embedding(dialogue_act)

        # Concatenate BERT output with Dialogue Act embeddings
        combined_output = torch.cat((pooled_output, dialogue_act_embedded), dim=1)

        # Pass through fully connected layer
        output = self.fc(self.dropout(combined_output))
        return output

In [11]:
# Training parameters
batch_size = 16
learning_rate = 5e-5
max_epochs = 3
n_splits = 5
patience = 2

In [13]:
# Initialize K-Fold cross-validator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_accuracies = []
best_accuracy = 0

In [14]:
# Start cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\nStarting Fold {fold + 1}/{n_splits}")
    
    # Split data for the current fold
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]
    dialogue_act_train_fold, dialogue_act_val_fold = dialogue_act_encoded[train_idx], dialogue_act_encoded[val_idx]

    # Tokenize the data
    train_encodings = tokenize_data(X_train_fold.astype(str).tolist())
    val_encodings = tokenize_data(X_val_fold.astype(str).tolist())

    # Prepare datasets and data loaders
    train_dataset = EmotionDataset(train_encodings, dialogue_act_train_fold, y_train_fold)
    val_dataset = EmotionDataset(val_encodings, dialogue_act_val_fold, y_val_fold)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    bert_model = BertModel.from_pretrained('bert-base-uncased')
    model = BertWithAdditionalFeatures(
        bert_model=bert_model,
        dialogue_act_vocab_size=len(label_encoder_dialogue_act.classes_),
        embedding_dim=16,
        num_labels=len(label_encoder_emotion.classes_)
    ).to(device)

    # Initialize optimizer and criterion
    optimizer = AdamW(model.parameters(), lr=learning_rate)  # AdamW from torch.optim
    criterion = nn.CrossEntropyLoss()

    # Early stopping initialization
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(max_epochs):
        print(f"\nEpoch {epoch + 1}/{max_epochs}")

        # Training phase
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                dialogue_act=batch['dialogue_act']
            )
            loss = criterion(outputs, batch['labels'])
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Calculate average training loss
        train_loss /= len(train_loader)
        print(f"Training Loss: {train_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0
        predictions, true_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    dialogue_act=batch['dialogue_act']
                )

                loss = criterion(outputs, batch['labels'])
                val_loss += loss.item()

                logits = outputs
                predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
                true_labels.extend(batch['labels'].cpu().numpy())

        # Calculate average validation loss and accuracy
        val_loss /= len(val_loader)
        accuracy = accuracy_score(true_labels, predictions)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.4f}")

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # Reset the patience counter
            best_model_state = model.state_dict()  # Save best model
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    # Load the best model state
    model.load_state_dict(best_model_state)
    fold_accuracies.append(accuracy)

    # Update best accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy


Starting Fold 1/5

Epoch 1/3


100%|██████████| 263/263 [01:26<00:00,  3.05it/s]


Training Loss: 0.4156
Validation Loss: 0.3988, Validation Accuracy: 0.8343

Epoch 2/3


100%|██████████| 263/263 [01:28<00:00,  2.98it/s]


Training Loss: 0.3125
Validation Loss: 0.3536, Validation Accuracy: 0.8552

Epoch 3/3


...

Training Loss: 0.4081
Validation Loss: 0.3672, Validation Accuracy: 0.8694

Epoch 2/3


100%|██████████| 263/263 [01:28<00:00,  2.97it/s]


Training Loss: 0.3033
Validation Loss: 0.4157, Validation Accuracy: 0.8513

Epoch 3/3


100%|██████████| 263/263 [01:28<00:00,  2.97it/s]


Training Loss: 0.2145
Validation Loss: 0.4570, Validation Accuracy: 0.8608
Early stopping triggered.


In [15]:
# Print cross-validation results
average_accuracy = np.mean(fold_accuracies)
print(f"\nCross-Validation Accuracy: {average_accuracy:.4f}")
print(f"Best Accuracy achieved: {best_accuracy:.4f}")


Cross-Validation Accuracy: 0.8544
Best Accuracy achieved: 0.8714


In [16]:
# Re-tokenize the full test set (assuming `X_test` and `y_test` are set aside initially)
X_test = data['Utterance'].values  # Replace with your actual test data
y_test = label_encoder_emotion.transform(data['Emotion'].values)  # Replace with actual test data labels
dialogue_act_test = label_encoder_dialogue_act.transform(data['Dialogue_Act'])  # Test set dialogue acts

In [17]:
# Tokenize the test set
test_encodings = tokenize_data(X_test.astype(str).tolist())
test_dataset = EmotionDataset(test_encodings, dialogue_act_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
 #Load the best-performing model state from cross-validation
model.load_state_dict(best_model_state)
model.to(device)

BertWithAdditionalFeatures(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [19]:
# Evaluate on test set
model.eval()
test_predictions, test_true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            dialogue_act=batch['dialogue_act']
        )

        logits = outputs
        test_predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        test_true_labels.extend(batch['labels'].cpu().numpy())

In [20]:
# Calculate test accuracy and classification report
test_accuracy = accuracy_score(test_true_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

target_names = [str(class_) for class_ in label_encoder_emotion.classes_]
test_report = classification_report(test_true_labels, test_predictions, target_names=target_names)
print(test_report)

Test Accuracy: 0.9421
              precision    recall  f1-score   support

          -1       0.90      0.87      0.88      1070
           0       0.95      0.98      0.97      4098
           1       1.00      0.04      0.07        79

    accuracy                           0.94      5247
   macro avg       0.95      0.63      0.64      5247
weighted avg       0.94      0.94      0.94      5247



In [69]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load BART model and tokenizer for summarization
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [71]:
def summarize_text(utterance):
    if len(utterance.split()) > 30:  
        inputs = bart_tokenizer(utterance, max_length=1024, return_tensors='pt', truncation=True)
        
        summary_ids = bart_model.generate(
            inputs['input_ids'], 
            max_length=60,
            min_length=25,
            length_penalty=2.0,
            num_beams=8,
            temperature=0.7,
            do_sample=True,      
            early_stopping=True
        )
        return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return utterance

In [72]:
def predict_emotion(utterance, dialogue_act):
    # Save the original utterance
    original_utterance = utterance

    # Summarize the utterance if it is too long (more than 30 words)
    summarized_utterance = summarize_text(utterance)

    # Tokenize and encode the summarized (or original) utterance
    utterance_encoding = tokenize_data([summarized_utterance])
    encoded_dialogue_act = torch.tensor(label_encoder_dialogue_act.transform([dialogue_act]), dtype=torch.long)

    # Move data to the appropriate device (GPU or CPU)
    input_ids = utterance_encoding['input_ids'].to(device)
    attention_mask = utterance_encoding['attention_mask'].to(device)
    dialogue_act = encoded_dialogue_act.to(device)

    # Perform model prediction
    model.eval()
    with torch.no_grad():
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            dialogue_act=dialogue_act
        )

        # Get the predicted emotion (from logits)
        predicted_emotion_idx = torch.argmax(output, dim=1).cpu().numpy()[0]
        predicted_emotion = label_encoder_emotion.inverse_transform([predicted_emotion_idx])[0]

    # Return both the original and summarized utterance along with the predicted emotion
    return original_utterance, summarized_utterance, predicted_emotion

In [73]:
# Example usage
new_sample = {
    "Utterance": "I’ve had a lot on my plate lately, both personally and professionally. I’ve been trying to stay on top of everything, but sometimes it feels like it’s just too much. My schedule has been packed, and I’ve had very little time to relax or take a break. I know I need to manage my time better and take care of my mental health, but it’s hard to find that balance. I feel like I’m constantly running from one task to the next.",
    "Dialogue_Act": "od"
}

In [74]:
original_utterance, summarized_utterance, predicted_emotion = predict_emotion(new_sample["Utterance"], new_sample["Dialogue_Act"])

In [75]:
# Display the results
print(f"Original Utterance: {original_utterance}")
print("\n")
print(f"Summarized Utterance: {summarized_utterance}")
print("\n")
print(f"Predicted Emotion: {predicted_emotion}")

Original Utterance: I’ve had a lot on my plate lately, both personally and professionally. I’ve been trying to stay on top of everything, but sometimes it feels like it’s just too much. My schedule has been packed, and I’ve had very little time to relax or take a break. I know I need to manage my time better and take care of my mental health, but it’s hard to find that balance. I feel like I’m constantly running from one task to the next.


Summarized Utterance: I’ve had a lot on my plate lately, both personally and professionally. I feel like I’m constantly running from one task to the next. I know I need to manage my time better and take care of my mental health.


Predicted Emotion: -1
